In [ ]:
import numpy as np
import cv2
import os


images_folder_with_fix_tail = "./N2-adults-50um-SRx5-fix-tail-crop-images-correct-angle-folders"
images_folder = './N2-adults-50um-SRx5-crop-images-correct-angle-folders'

seg_folder_with_fix_tail = "./N2-adults-50um-SRx5-fix-tail-crop-seg-correct-angle-folders"
seg_folder = './N2-adults-50um-SRx5-crop-seg-correct-angle-folders'


sub_folders = os.listdir(images_folder)

print(sub_folders)

pointcloud_folder = "N2-adults-50um-SRx5-3D-pointcloud"





In [ ]:
import math
import open3d
from copy import deepcopy
import pandas as pd


for sub_folder in sub_folders:

    images_with_fix_tail_list_dir = images_folder_with_fix_tail+'/'+sub_folder
    seg_with_fix_tail_images_list_dir = seg_folder_with_fix_tail+'/'+sub_folder


    images_list_dir = images_folder+'/'+sub_folder
    seg_images_list_dir = seg_folder+'/'+sub_folder

    images_list = os.listdir(images_list_dir)
    seg_images_list = os.listdir(seg_images_list_dir)


    if sub_folder not in ['2nd_']: 
        continue
    print(sub_folder)

    
    best_match_index = 0
    best_match_score = 0
    for image_name in images_list:

        if image_name.endswith('.png'):
            image_index = int(image_name.replace('.png',''))
            
            if image_index==0:

                img_tempate = cv2.imread(images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE)

            elif image_index>100:
                img = cv2.imread(images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE)

                res = cv2.matchTemplate(img, img_tempate, cv2.TM_CCOEFF_NORMED)

                score = res[0][0]
                if score > best_match_score:
                    best_match_index = image_index
                    best_match_score = score
                    

    print('best_match_index = ',best_match_index, ', best_match_score = ' ,best_match_score)

    

    
    pcd_offset_file = open3d.io.read_point_cloud(pointcloud_folder+'/'+sub_folder+'/best_iou_after_offset.pcd')


    
    degree_per_frame = 360.0/(best_match_index+1)
    origin_pcd = deepcopy(pcd_offset_file)
    pcd2 = deepcopy(origin_pcd)

    N = len(pcd2.points)
    red_color = [1.0, 1.0, 1.0]  
    colors = np.tile(red_color, (N, 1))
    pcd2.colors = open3d.utility.Vector3dVector(colors)

    for image_name in images_list:

        if image_name.endswith('.png'):
            image_index = int(image_name.replace('.png',''))
            

            
            


            
            if image_index >= 0 and image_index <= best_match_index:
                
                img  = cv2.imread(images_with_fix_tail_list_dir + '/' + image_name) 
                mask = cv2.imread(seg_with_fix_tail_images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE) 


                
                h,w = mask.shape
                contours = cv2.findContours(
                    mask.astype("uint8"), cv2.RETR_CCOMP,
                    cv2.CHAIN_APPROX_NONE)[-2]

                c_index = 0
                for i in range(len(contours)):

                    mu=cv2.moments(contours[i],False)
                    area = cv2.contourArea(contours[i])
                    if area > 100000:
                        c_index = i

                
                contours_by_height = {}
                
                for i in range(h):
                    contours_by_height[i] = []

                    condition = (contours[c_index][:, :, 1] == i)  
                    filtered_data = contours[c_index][condition]  
                    contours_by_height[i] = filtered_data

            
                xyz_point_center_line = []
                for data_y in contours_by_height:

                    
                    length = len(contours_by_height[data_y])

                    
                    if length > 0:
                        if length == 1:
                            width = 1
                        else:

                            
                            x_coords = contours_by_height[data_y][:,0]
                            
                            max_x = np.max(x_coords)
                            min_x = np.min(x_coords)
                            width = max_x - min_x

                            r = width/2.0 
                        
                            x = int(min_x + r)
                            y = int(data_y)
                            xyz_point_center_line.append([x,y])

                            for kk in range(1,6):
                                xyz_point_center_line.append([int(x+kk),y])
                                xyz_point_center_line.append([int(x-kk),y])

                            

                

                mu=cv2.moments(contours[c_index],False)
                mask_center_x = int(mu['m10'] / (mu['m00']+0.00000001))
                mask_center_y = int(mu['m01'] / (mu['m00']+0.00000001))
                mask_x, mask_y, mask_w, mask_h = cv2.boundingRect(contours[c_index])
                mask_bottom_y = mask_y + mask_h



                R = pcd2.get_rotation_matrix_from_xyz((0, -math.radians(degree_per_frame * image_index), 0))
                pcd2.rotate(R)
                pcd_offset_points = np.asarray(pcd2.points).astype(np.int32)
                pcd_offset_projection_points = pcd_offset_points[:,0:2]
                pcd_offset_projection_points[:,0] += 239
                projection_image = np.zeros([2600,480])
                pcd_offset_projection_points = np.unique(pcd_offset_projection_points, axis=0)
                projection_image[pcd_offset_projection_points[:, 1], pcd_offset_projection_points[:, 0]] = 255




                
                contours_prj = cv2.findContours(
                    projection_image.astype("uint8"), cv2.RETR_CCOMP,
                    cv2.CHAIN_APPROX_NONE)[-2]

                prj_c_index = 0
                for i in range(len(contours_prj)):

                    mu=cv2.moments(contours_prj[i],False)
                    area = cv2.contourArea(contours_prj[i])
                    if area > 50000:
                        prj_c_index = i

                
                projection_image = np.zeros_like(projection_image)
                cv2.fillPoly(projection_image, [contours_prj[prj_c_index]], (255))   


                mu=cv2.moments(contours_prj[prj_c_index],False)
                projection_image_center_x = int(mu['m10'] / (mu['m00']+0.00000001))
                projection_image_center_y = int(mu['m01'] / (mu['m00']+0.00000001))
                projection_image_x, projection_image_y, projection_image_w, projection_image_h = cv2.boundingRect(contours[c_index])
                projection_image_bottom_y = projection_image_y + projection_image_h


                
                tx = mask_center_x-projection_image_center_x 
                
                ty = mask_bottom_y-projection_image_bottom_y 

                affine_arr = np.float32([[1,0,tx],[0,1,ty]])
                projection_image = cv2.warpAffine(projection_image,affine_arr,(projection_image.shape[1],projection_image.shape[0]))  


                
                
                


                
                projection_list = []
                projection = np.zeros_like(img)

                

                

                
                
                
                

                
                
                
                

                
                

                
                
                

                
                

                
                
                

                

                
                
                

                points_array = np.array(pcd2.points)
                colors_array = np.array(pcd2.colors)

                
                x_values, y_values, z_values = points_array[:, 0], points_array[:, 1], points_array[:, 2]

                
                x_values = (x_values + 239 + tx).astype(int)
                y_values = (y_values + ty).astype(int)
                z_values = z_values.astype(int)

                
                df_xyz_point_center_line = pd.DataFrame(xyz_point_center_line, columns=['x', 'y'])
                df_points = pd.DataFrame(np.vstack((x_values, y_values)).T, columns=['x', 'y'])

                
                df_points['index'] = df_points.index

                
                df_merged = pd.merge(df_points, df_xyz_point_center_line, how='left', indicator=True)

                
                mask_indices = (df_merged['_merge'] == 'both') & (z_values > 0)

                colors_array[df_merged.loc[mask_indices, 'index']] = img[y_values[mask_indices], x_values[mask_indices]] / 255
                pcd2.colors = open3d.utility.Vector3dVector(colors_array)

                
                index2 = (x_values >= 0) & (x_values < 480) & (y_values >= 0) & (y_values < 2600) & (z_values >= 0)

                
                projection[y_values[index2], x_values[index2]] = 255


                


                

                
                
                
                
                
                    
                
                

                

                
                

                

                

                
                
                
                

                

                
                R = pcd2.get_rotation_matrix_from_xyz((0, math.radians(degree_per_frame * image_index), 0))
                pcd2.rotate(R)

    
    
    open3d.io.write_point_cloud(pointcloud_folder+'/'+sub_folder+'/cloud_centerline_origin.ply', origin_pcd , write_ascii=True)  
    open3d.io.write_point_cloud(pointcloud_folder+'/'+sub_folder+'/cloud_centerline_tietu.ply', pcd2 , write_ascii=True)  